In [1]:
%run -i utilities.py

In [2]:
# Set file paths and load files
file_path = Path(
    'eeg-data/Stephanie/Rew_601_rest/Rew_601_rest_bb_epoch.set')
mat_reject = Path(
    'eeg-data/Stephanie/Rew_601_rest/Rew_601_rest_reject_rmm.mat')
mat_stage = Path(
    'eeg-data/Stephanie/Rew_601_rest/Rew_601_rest_stages.mat')

files = load_subject_dir(file_path, mat_reject, mat_stage)

At least one epoch has multiple events. Only the latency of the first event will be retained.
Extracting parameters from eeg-data\Stephanie\Rew_601_rest\Rew_601_rest_bb_epoch.set...


C:\Users\peter\git\EEG-artifact-rejection\artifact-rejection\utilities.py:30: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  set_file = mne.io.read_epochs_eeglab(file_path)


6476 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.


In [3]:
epochs = files['epochs']
try:
    reject = files['reject']
except:
    pass

In [4]:
# Read epochs into dataframe and clean df
index, scaling_time, scalings = ['epoch', 'time'], 1e3, dict(grad=1e13)
df_read = epochs.to_data_frame(
    picks=None, scalings=scalings, scaling_time=scaling_time, index=index)
df = clean_df(df_read)
df_ = df.copy()

Converting time column to int64...
Cleaning data...
Cleaned data successfully!



In [ ]:
# Select values from dataframe and reject file
df_values = extract_df_values(df)
X, y = df_values.values, reject
df_IF = run_IForest(X, y, df)

In [ ]:
# Create .csv with rejected <#> epochs
drop_epochs = sorted(list(set(df_IF['epoch'])))
drop_epochs_ = np.asarray(drop_epochs)
drop_epochs_.tofile('pred_y.csv', sep=',', format='%1.0f')

In [5]:
pred_y_file = np.genfromtxt('pred_y.csv', delimiter=',')

In [6]:
reject_epochs = [index for index, value in enumerate(reject) if value==1]

In [7]:
np.array([int(x) for x in pred_y_file if x not in reject_epochs])

array([ 156,  164,  188,  221,  241,  256,  257,  261,  288,  298,  301,
        302,  306,  318,  335,  339,  341,  355,  358,  362,  381,  405,
        415,  423,  431,  441,  445,  456,  462,  498,  516,  527,  539,
        551,  558,  564,  567,  574,  592,  619,  628,  641,  647,  648,
        650,  654,  661,  662,  666,  681,  700,  704,  901,  964,  968,
        983, 1000, 1002, 1009, 1033, 1038, 1040, 1059, 1073, 1074, 1080,
       1086, 1088, 1108, 1114, 1141, 1167, 1202, 1206, 1207, 1213, 1218,
       1230, 1245, 1250, 1257, 1259, 1264, 1290, 1340, 1395, 1403, 1410,
       1419, 1429, 1432, 1455, 1466, 1467, 1474, 1483, 1511, 1520, 1550,
       1559, 1579, 1588, 1597, 1614, 1618, 1621, 1625, 1631, 1634, 1639,
       1646, 1905, 1910, 1921, 1934, 1947, 2063, 2110, 2131, 2150, 2158,
       2197, 2200, 2205, 2240, 2281, 2392, 2426, 2459, 2476, 2503, 2512,
       2542, 2545, 2548, 2551, 2566, 2570, 2581, 2666, 2676, 2692, 2702,
       2719, 2726, 2754, 2776, 2778, 2790, 3114, 31

In [8]:
np.array([int(x) for x in pred_y_file if x in reject_epochs])

array([  95,  307,  308,  309,  366,  367,  393,  394,  395,  469,  470,
        471,  473,  474,  555,  556,  560,  561,  759, 1095, 1096, 1097,
       1098, 1099, 1100, 1106, 1107, 1297, 1298, 1351, 1363, 1364, 1608,
       1609, 1610, 1869, 1870, 1991, 1992, 1993, 2180, 2181, 2182, 2258,
       2259, 2283, 2285, 2311, 2821, 2822, 3070, 3071, 3072, 3269, 3270,
       3285, 3286, 3287, 3430, 3558, 3559, 3560, 3561, 3819, 3820, 3916,
       3917, 3918, 4056, 4057, 4122, 4123, 4124, 4440, 4645, 4848, 4849,
       4920, 4921, 5150, 5347, 5366, 5367, 5368, 5407, 5408, 5409, 5434,
       5435, 5511, 5512, 5513, 5515, 5516, 5671, 5672, 5766, 5767, 5886,
       5887, 5888, 5950, 5951, 5952, 6057, 6058, 6315, 6316, 6347, 6348,
       6409])

In [9]:
len(reject_epochs)

407

In [10]:
len(np.array([int(x) for x in pred_y_file if x in reject_epochs]))

111

In [16]:
df_bool = (df_ == df)

In [19]:
np.count_nonzero(df_bool.iloc[:, 1])

12952000

In [20]:
df.head()

signal,time,epoch,C3,C4,CZ,F3,F4,F7,F8,FZ,...,Fp2,O1,O2,P3,P4,PZ,T3,T4,T5,T6
0,0,0,-4.101171,-5.968149,-4.743762,-3.682605,-4.446309,-1.441384,-5.157160,-2.841753,...,-5.491191,-5.813361,-10.619266,-5.897866,-7.994251,-6.512607,-2.072411,-2.857092,-1.348035,-6.712998
1,2,0,-5.183039,-3.539983,-4.891985,-5.510036,-2.731580,-4.127728,-2.581909,-2.712256,...,-4.066828,-8.367174,-10.431769,-7.622222,-6.328388,-7.438750,-3.432891,-0.377738,-3.379942,-5.844828
2,5,0,-6.473076,-1.935451,-5.402406,-7.127059,-1.392381,-6.066678,-0.504993,-2.933984,...,-2.699850,-10.873877,-10.933600,-9.510497,-5.507238,-8.706780,-4.636084,1.354425,-5.225135,-5.669819
3,8,0,-8.008626,-1.666145,-6.403526,-8.324136,-0.666902,-6.764383,0.725579,-3.599645,...,-1.478360,-13.169887,-12.535002,-11.537941,-6.059763,-10.455081,-5.518118,1.881818,-6.697570,-6.624831
4,10,0,-9.597174,-2.770913,-7.707548,-8.924362,-0.604117,-6.133142,0.986625,-4.461777,...,-0.552566,-14.950358,-15.219418,-13.436047,-8.015812,-12.517210,-5.916834,1.181734,-7.626414,-8.737988


In [23]:
new_index = [2,4,6]
df.loc[[2, 4, 6],'epoch'] = 1010101

In [25]:
df.loc[[2, 4, 6],'epoch']

2    1010101
4    1010101
6    1010101
Name: epoch, dtype: int32